# LNDL Resolver - Tool Resolution & Registry Management

The LNDL (Lion Directive Language) resolver is the core validation and execution engine that:

**Core Features:**
- **Reference Resolution**: Maps variable/action names to values using namespace-prefixed declarations
- **Type Validation**: Validates outputs against Operable specs (strict type checking)
- **Action Lifecycle**: Manages three-phase action execution (parse → execute → revalidate)
- **Mixed Inputs**: Supports combining static lvars with dynamic lacts in BaseModel construction
- **Error Aggregation**: Collects validation errors into ExceptionGroups for batch reporting

**Key Functions:**
- `parse_lndl()`: High-level API - parses full LLM response and validates
- `resolve_references_prefixed()`: Low-level resolver - validates OUT{} block against specs

In [1]:
from pydantic import BaseModel, Field

from lionherd_core.lndl import (
    ActionCall,
    ensure_no_action_calls,
    has_action_calls,
    parse_lndl,
    resolve_references_prefixed,
    revalidate_with_action_results,
)
from lionherd_core.lndl.errors import MissingFieldError, TypeMismatchError
from lionherd_core.types import Operable, Spec

## 1. Basic Resolution - Static Variables Only

Start with the simplest case: resolving static lvar declarations without actions.

In [2]:
# Define a simple model
class Report(BaseModel):
    title: str
    score: float


# Create Operable with Report spec
operable = Operable(specs=[Spec(name="report", base_type=Report)])

# Mock LNDL response with lvars
response = """
<lvar Report.title t>Quarterly Analysis</lvar>
<lvar Report.score s>0.92</lvar>

OUT{
    report: [t, s]
}
"""

# Parse and resolve
output = parse_lndl(response, operable)

print(f"Report: {output.report}")
print(f"Title: {output.report.title}")
print(f"Score: {output.report.score}")
print(f"Type: {type(output.report).__name__}")

Report: title='Quarterly Analysis' score=0.92
Title: Quarterly Analysis
Score: 0.92
Type: Report


## 2. LNDLOutput Structure

Understanding the output object's structure and access patterns.

In [3]:
# Access fields multiple ways
print("Access patterns:")
print(f"1. Attribute: {output.report}")
print(f"2. Dict-style: {output['report']}")
print(f"3. Fields dict: {output.fields['report']}")

# Inspect metadata
print(f"\nLvars declared: {list(output.lvars.keys())}")
print(f"Lacts declared: {list(output.lacts.keys())}")
print(f"Actions referenced in OUT{{}}: {list(output.actions.keys())}")

# Check lvar metadata
lvar_t = output.lvars["t"]
print("\nLvar 't' metadata:")
print(f"  Model: {lvar_t.model}")
print(f"  Field: {lvar_t.field}")
print(f"  Value: {lvar_t.value}")

Access patterns:
1. Attribute: title='Quarterly Analysis' score=0.92
2. Dict-style: title='Quarterly Analysis' score=0.92
3. Fields dict: title='Quarterly Analysis' score=0.92

Lvars declared: ['t', 's']
Lacts declared: []
Actions referenced in OUT{}: []

Lvar 't' metadata:
  Model: Report
  Field: title
  Value: Quarterly Analysis


## 3. Scalar Field Resolution

Scalar fields (str, int, float, bool) have special handling - single variable or literal assignment.

In [4]:
# Operable with scalar specs
operable_scalars = Operable(
    specs=[
        Spec(name="title", base_type=str),
        Spec(name="count", base_type=int),
        Spec(name="ratio", base_type=float),
        Spec(name="active", base_type=bool),
    ]
)

response = """
OUT{
    title: "Direct Literal",
    count: 42,
    ratio: 0.75,
    active: true
}
"""

output = parse_lndl(response, operable_scalars)

print(f"Title: {output.title} (type: {type(output.title).__name__})")
print(f"Count: {output.count} (type: {type(output.count).__name__})")
print(f"Ratio: {output.ratio} (type: {type(output.ratio).__name__})")
print(f"Active: {output.active} (type: {type(output.active).__name__})")

Title: Direct Literal (type: str)
Count: 42 (type: int)
Ratio: 0.75 (type: float)
Active: True (type: bool)


## 4. Required vs Optional Fields

Specs can be marked as required (default) or optional.

In [5]:
# Operable with required and optional fields
operable_mixed = Operable(
    specs=[
        Spec(name="required_field", base_type=str, required=True),
        Spec(name="optional_field", base_type=str, required=False),
    ]
)

# Valid: required field present, optional missing
response_valid = """
OUT{
    required_field: "Present"
}
"""

output = parse_lndl(response_valid, operable_mixed)
print(f"Valid: required_field = {output.required_field}")
print(f"Optional field in output: {'optional_field' in output.fields}")

# Invalid: required field missing
response_invalid = """
OUT{
    optional_field: "Only optional"
}
"""

try:
    parse_lndl(response_invalid, operable_mixed)
except MissingFieldError as e:
    print(f"\n✓ Missing required field caught: {e}")

Valid: required_field = Present
Optional field in output: False

✓ Missing required field caught: Required field 'required_field' missing from OUT{}


## 5. Action Lifecycle - Three Phases

Actions follow a strict lifecycle: **Parse → Execute → Revalidate**

Phase 1 (Parse): ActionCall objects created, partial validation only

In [6]:
class SearchResult(BaseModel):
    query: str
    results: int = Field(gt=0)  # Must be positive


operable_action = Operable(specs=[Spec(name="search", base_type=SearchResult)])

response_with_action = """
<lvar SearchResult.query q>AI research</lvar>
<lact SearchResult.results r>search_api(query="AI", limit=10)</lact>

OUT{
    search: [q, r]
}
"""

# Phase 1: Parse - creates ActionCall placeholders
output = parse_lndl(response_with_action, operable_action)

print("Phase 1: Parse complete")
print(f"Search result type: {type(output.search).__name__}")
print(f"Query (static): {output.search.query}")
print(f"Results (ActionCall): {output.search.results}")
print(f"Results type: {type(output.search.results).__name__}")

# Check for actions
print(f"\nHas action calls: {has_action_calls(output.search)}")
print(f"Actions to execute: {list(output.actions.keys())}")

Phase 1: Parse complete
Search result type: SearchResult
Query (static): AI research
Results (ActionCall): ActionCall(name='r', function='search_api', arguments={'query': 'AI', 'limit': 10}, raw_call='search_api(query="AI", limit=10)')
Results type: ActionCall

Has action calls: True
Actions to execute: ['r']


**Phase 2 (Execute)**: Caller executes actions using the `.actions` dict

In [7]:
# Mock action execution
def execute_action(action: ActionCall) -> int:
    """Mock tool executor - returns count based on limit."""
    print(f"Executing: {action.function}({action.arguments})")
    limit = action.arguments.get("limit", 5)
    return limit  # Simulate API returning result count


# Execute all actions
action_results = {}
for name, action in output.actions.items():
    result = execute_action(action)
    action_results[name] = result

print("\nPhase 2: Execution complete")
print(f"Action results: {action_results}")

Executing: search_api({'query': 'AI', 'limit': 10})

Phase 2: Execution complete
Action results: {'r': 10}


**Phase 3 (Revalidate)**: Replace ActionCall objects with results and run full Pydantic validation

In [8]:
# Phase 3: Revalidate with action results
validated_search = revalidate_with_action_results(output.search, action_results)

print("Phase 3: Revalidation complete")
print(f"Query: {validated_search.query}")
print(f"Results: {validated_search.results} (type: {type(validated_search.results).__name__})")
print(f"Has action calls: {has_action_calls(validated_search)}")

# Now safe for persistence
final = ensure_no_action_calls(validated_search)
print(f"\n✓ Safe for database: {final}")

Phase 3: Revalidation complete
Query: AI research
Results: 10 (type: int)
Has action calls: False

✓ Safe for database: query='AI research' results=10


/Users/lion/projects/open-source/lionherd-core/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [field_name='results', input_value=ActionCall(name='r', func...(query="AI", limit=10)'), input_type=ActionCall])
  return self.__pydantic_serializer__.to_python(


## 6. Direct Actions - Returning Entire Models

Direct actions (no namespace) can return complete BaseModel instances.

In [9]:
class Document(BaseModel):
    title: str
    content: str
    word_count: int


operable_doc = Operable(specs=[Spec(name="document", base_type=Document)])

# Direct action returns entire Document
response_direct = """
<lact fetch_doc>fetch_document(doc_id="report-123")</lact>

OUT{
    document: [fetch_doc]
}
"""

output = parse_lndl(response_direct, operable_doc)

print("Direct action parsed:")
print(f"Document type: {type(output.document).__name__}")
print(f"Action to execute: {output.actions['fetch_doc']}")

# Mock execution returning full model
mock_document = Document(title="Q4 Report", content="Full text...", word_count=1500)
action_results = {"fetch_doc": mock_document}

# The direct action case is simpler - just replace the field value
output.fields["document"] = action_results["fetch_doc"]

print("\nAfter execution:")
print(f"Document: {output.document}")
print(f"Title: {output.document.title}")

Direct action parsed:
Document type: ActionCall
Action to execute: ActionCall(name='fetch_doc', function='fetch_document', arguments={'doc_id': 'report-123'}, raw_call='fetch_document(doc_id="report-123")')

After execution:
Document: title='Q4 Report' content='Full text...' word_count=1500
Title: Q4 Report


## 7. Mixing Lvars and Lacts

Powerful pattern: combine static values with dynamic actions in the same model.

In [10]:
class Analysis(BaseModel):
    dataset: str  # Static
    method: str  # Static
    accuracy: float  # Dynamic (from action)
    confidence: float  # Dynamic (from action)


operable_mixed = Operable(specs=[Spec(name="analysis", base_type=Analysis)])

response_mixed = """
<lvar Analysis.dataset d>customer_churn</lvar>
<lvar Analysis.method m>random_forest</lvar>
<lact Analysis.accuracy acc>evaluate_model(dataset="customer_churn")</lact>
<lact Analysis.confidence conf>calculate_confidence(model="rf")</lact>

OUT{
    analysis: [d, m, acc, conf]
}
"""

output = parse_lndl(response_mixed, operable_mixed)

print("Mixed lvar/lact construction:")
print(f"Dataset (static): {output.analysis.dataset}")
print(f"Method (static): {output.analysis.method}")
print(f"Accuracy (action): {output.analysis.accuracy}")
print(f"Confidence (action): {output.analysis.confidence}")

# Execute actions
action_results = {"acc": 0.87, "conf": 0.92}
validated_analysis = revalidate_with_action_results(output.analysis, action_results)

print("\nAfter revalidation:")
print(f"Accuracy: {validated_analysis.accuracy}")
print(f"Confidence: {validated_analysis.confidence}")

Mixed lvar/lact construction:
Dataset (static): customer_churn
Method (static): random_forest
Accuracy (action): ActionCall(name='acc', function='evaluate_model', arguments={'dataset': 'customer_churn'}, raw_call='evaluate_model(dataset="customer_churn")')
Confidence (action): ActionCall(name='conf', function='calculate_confidence', arguments={'model': 'rf'}, raw_call='calculate_confidence(model="rf")')

After revalidation:
Accuracy: 0.87
Confidence: 0.92


/Users/lion/projects/open-source/lionherd-core/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [field_name='accuracy', input_value=ActionCall(name='acc', fu...aset="customer_churn")'), input_type=ActionCall])
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [field_name='confidence', input_value=ActionCall(name='conf', f...confidence(model="rf")'), input_type=ActionCall])
  return self.__pydantic_serializer__.to_python(


## 8. Error Handling - Type Mismatches

The resolver validates that variable/action namespaces match the expected model type.

In [11]:
class User(BaseModel):
    name: str
    email: str


class Product(BaseModel):
    title: str
    price: float


operable_mismatch = Operable(specs=[Spec(name="user", base_type=User)])

# Wrong model namespace
response_wrong = """
<lvar Product.title t>Laptop</lvar>
<lvar User.name n>Alice</lvar>

OUT{
    user: [t, n]
}
"""

try:
    parse_lndl(response_wrong, operable_mismatch)
except* TypeMismatchError as eg:
    print("✓ Type mismatch caught:")
    for exc in eg.exceptions:
        print(f"  {exc}")

✓ Type mismatch caught:
  Variable 't' is for model 'Product', but field 'user' expects 'User'


## 9. Error Aggregation - ExceptionGroups

Multiple validation errors are collected and reported together.

In [12]:
operable_multi = Operable(
    specs=[
        Spec(name="user", base_type=User),
        Spec(name="product", base_type=Product),
    ]
)

# Multiple errors: missing variable + type mismatch
response_multi_error = """
<lvar User.name n>Bob</lvar>
<lvar Product.title t>Phone</lvar>

OUT{
    user: [n, missing_email],
    product: [t, n]
}
"""

try:
    parse_lndl(response_multi_error, operable_multi)
except ExceptionGroup as eg:
    print(f"✓ Caught {len(eg.exceptions)} errors:")
    for i, exc in enumerate(eg.exceptions, 1):
        print(f"  {i}. {type(exc).__name__}: {exc}")

✓ Caught 2 errors:
  1. ValueError: Variable or action 'missing_email' referenced in OUT{} but not declared
  2. TypeMismatchError: Variable 'n' is for model 'User', but field 'product' expects 'Product'


## 10. Name Collision Detection

Lvar and lact names must be unique - no collisions allowed.

In [13]:
response_collision = """
<lvar Report.title t>Title from lvar</lvar>
<lact Report.score t>calculate_score()</lact>

OUT{
    report: [t]
}
"""

operable_collision = Operable(specs=[Spec(name="report", base_type=Report)])

try:
    parse_lndl(response_collision, operable_collision)
except ValueError as e:
    print(f"✓ Collision detected: {e}")

✓ Collision detected: Name collision detected: {'t'} used in both <lvar> and <lact> declarations


## 11. Ensure No Action Calls - Safety Guard

Critical guard before persisting models to prevent database corruption.

In [14]:
# Create model with unexecuted actions
response_guard = """
<lvar Report.title t>Unvalidated Report</lvar>
<lact Report.score s>calculate_score()</lact>

OUT{
    report: [t, s]
}
"""

output = parse_lndl(response_guard, operable)

# Attempt to save without executing actions
try:
    ensure_no_action_calls(output.report)  # Guard prevents corruption
except ValueError as e:
    print(f"✓ Guard blocked persistence: {e}")

# Proper flow: execute then save
action_results = {"s": 0.95}
validated_report = revalidate_with_action_results(output.report, action_results)
safe_report = ensure_no_action_calls(validated_report)
print(f"\n✓ Safe to persist: {safe_report}")

✓ Guard blocked persistence: Report contains unexecuted actions in fields: score. Models with ActionCall placeholders must be re-validated after action execution. Call revalidate_with_action_results() before using this model.

✓ Safe to persist: title='Unvalidated Report' score=0.95


/Users/lion/projects/open-source/lionherd-core/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [field_name='score', input_value=ActionCall(name='s', func...all='calculate_score()'), input_type=ActionCall])
  return self.__pydantic_serializer__.to_python(


## 12. Scalar Actions

Actions can return scalar values directly for non-BaseModel fields.

In [15]:
operable_scalar_action = Operable(
    specs=[
        Spec(name="temperature", base_type=float),
        Spec(name="status", base_type=str),
    ]
)

response_scalar_action = """
<lact temp>get_temperature(sensor="living_room")</lact>
<lact stat>check_status(system="hvac")</lact>

OUT{
    temperature: [temp],
    status: [stat]
}
"""

output = parse_lndl(response_scalar_action, operable_scalar_action)

print("Scalar actions parsed:")
print(f"Temperature: {output.temperature} (type: {type(output.temperature).__name__})")
print(f"Status: {output.status} (type: {type(output.status).__name__})")

# These are ActionCall objects before execution
print(f"\nActions to execute: {list(output.actions.keys())}")

# Execute and replace
output.fields["temperature"] = 72.5
output.fields["status"] = "normal"

print("\nAfter execution:")
print(f"Temperature: {output.temperature}")
print(f"Status: {output.status}")

Scalar actions parsed:
Temperature: ActionCall(name='temp', function='get_temperature', arguments={'sensor': 'living_room'}, raw_call='get_temperature(sensor="living_room")') (type: ActionCall)
Status: ActionCall(name='stat', function='check_status', arguments={'system': 'hvac'}, raw_call='check_status(system="hvac")') (type: ActionCall)

Actions to execute: ['temp', 'stat']

After execution:
Temperature: 72.5
Status: normal


## 13. Low-Level API - resolve_references_prefixed

For custom parsing workflows, use the low-level resolver directly.

In [16]:
from lionherd_core.lndl.parser import (
    extract_lacts_prefixed,
    extract_lvars_prefixed,
    extract_out_block,
    parse_out_block_array,
)

response_lowlevel = """
<lvar Report.title t>Custom Parsing</lvar>
<lvar Report.score s>0.88</lvar>

OUT{
    report: [t, s]
}
"""

# Step 1: Extract components
lvars = extract_lvars_prefixed(response_lowlevel)
lacts = extract_lacts_prefixed(response_lowlevel)
out_content = extract_out_block(response_lowlevel)
out_fields = parse_out_block_array(out_content)

print("Extracted components:")
print(f"Lvars: {list(lvars.keys())}")
print(f"Lacts: {list(lacts.keys())}")
print(f"OUT fields: {out_fields}")

# Step 2: Resolve references
output = resolve_references_prefixed(out_fields, lvars, lacts, operable)

print(f"\nResolved report: {output.report}")

Extracted components:
Lvars: ['t', 's']
Lacts: []
OUT fields: {'report': ['t', 's']}

Resolved report: title='Custom Parsing' score=0.88


## 14. Complex Nested Models

Resolution works with deeply nested Pydantic models.

In [17]:
class Author(BaseModel):
    name: str
    email: str


class Article(BaseModel):
    title: str
    author: Author
    views: int


# Note: LNDL currently resolves one level - nested models need separate specs
# or you use direct actions returning complete nested structures
operable_nested = Operable(
    specs=[
        Spec(name="author", base_type=Author),
        Spec(name="article_title", base_type=str),
        Spec(name="article_views", base_type=int),
    ]
)

response_nested = """
<lvar Author.name n>Alice Smith</lvar>
<lvar Author.email e>alice@example.com</lvar>

OUT{
    author: [n, e],
    article_title: "Understanding LNDL",
    article_views: 1500
}
"""

output = parse_lndl(response_nested, operable_nested)

# Manually construct Article from resolved parts
article = Article(title=output.article_title, author=output.author, views=output.article_views)

print(f"Article: {article}")
print(f"Author name: {article.author.name}")
print(f"Author email: {article.author.email}")

Article: title='Understanding LNDL' author=Author(name='Alice Smith', email='alice@example.com') views=1500
Author name: Alice Smith
Author email: alice@example.com


## Summary Checklist

**LNDL Resolver Essentials:**
- ✅ Resolves namespace-prefixed lvars/lacts against Operable specs
- ✅ Validates type compatibility (strict matching)
- ✅ Three-phase action lifecycle (parse → execute → revalidate)
- ✅ Supports mixing static lvars with dynamic lacts
- ✅ Aggregates validation errors into ExceptionGroups
- ✅ Detects name collisions between lvars and lacts
- ✅ Guards against persisting unexecuted actions
- ✅ Handles both scalar and BaseModel field types
- ✅ Direct actions can return complete models
- ✅ Required/optional field validation

**Key Safety Rules:**
1. **Always revalidate** after executing actions
2. **Use ensure_no_action_calls()** before persistence
3. **Check has_action_calls()** to determine if execution needed
4. **Handle ExceptionGroups** for comprehensive error reporting

**Next Steps:**
- See `lndl_fuzzy.ipynb` for fuzzy matching and error recovery
- See `types_operable.ipynb` for Operable/Spec details
- See `schema_function_call_parser.ipynb` for action parsing